This notebook can be run on the Copernicus Dataspace Jupyterhub but running the following package installation cell first

**Note** You should select on of the kernels with GDAL installed, eg. "Geo science"

In [1]:
try:
    import eomaji
except ModuleNotFoundError:
    !pip install eomaji@git+https://github.com/DHI/EOMAJI-OpenEO-toolbox.git

# Two-Source Energy Balance (TSEB) Model Implementation
This notebook implements the **Two-Source Energy Balance (TSEB) model** using the `pyTSEB` package.
It processes sharpened Sentinel-3 Land Surface Temperature (LST) data along with meteorological and vegetation parameters
to compute land surface energy fluxes at high spatial resolution.

In [7]:
import hashlib
from pathlib import Path
from pyTSEB.PyTSEB import PyTSEB
from eomaji.utils.general_utils import read_area_date_info

## 1. Define AOI and date
Either read it from dumped information from the [prepare_data.ipynb](notebooks/prepare_data.ipynb) notebook, or define it yourself

In [8]:
date_dir = "./data"
date, bbox = read_area_date_info(
    dir=date_dir
)
#date = datetime.date(2023, 6, 25)
#bbox = [6.153142, 45.045924, 6.433234, 45.251259]

### Define paths for input and output data

In [28]:
date_str = str(date).replace("-", "")
bbox_hash = hashlib.md5(str(bbox).encode()).hexdigest()[:8]
base_data_path = Path(date_dir) / f"{date_str}_{bbox_hash}"
meteorological_data_path = base_data_path / "meteo_data"
output_file = base_data_path / f"{date_str}_image.vrt"
date_time = next((f for f in meteorological_data_path.glob("*") if f.is_file() and "NIR" in f.stem), None).stem.split('_')[0]

### Define TSEB model parameters
You can leave it as it is if you where running the other notebooks with default values

In [34]:
params = {
    "model": "TSEB_PT",
    "output_file": str(output_file),
    "T_R1": str(base_data_path / "sharpened_LST.tif"),  # land surface temperature - this should be the Sharpened Sentinel-3 LST
    "VZA": str(base_data_path / f"{date_str}_VZA.tif"),
    "input_mask": 0,
    "LAI": str(base_data_path / f"{date_str}_LAI.tif"),
    "f_c": str(base_data_path / f"{date_str}_FCOVER.tif"),
    "h_C": str(base_data_path / f"{date_str}_H_C.tif"),
    "w_C": str(base_data_path / f"{date_str}_W_C.tif"),
    "f_g": str(base_data_path / f"{date_str}_F_G.tif"),
    "lat": 38.289355,  # <INPUT_DATASET>_LAT.tif
    "lon": -121.117794,  # <INPUT_DATASET>_LONG.tif
    "alt": str(base_data_path / f"{date_str}_ELEV.tif"),
    "stdlon": 0,
    "z_T": 5,
    "z_u": 5,
    "DOY": 221,  # <INPUT_DATASET>_DOY_UTC.tif
    "time": 10.9992,  # <INPUT_DATASET>_TIME_UTC.tif
    "T_A1": str(meteorological_data_path / f"{date_time}_T_A1.tif"),
    "u": str(meteorological_data_path / f"{date_time}_u.tif"),
    "p": str(meteorological_data_path / f"{date_time}_p.tif"),
    "ea": str(meteorological_data_path / f"{date_time}_EA.tif"),
    "S_dn": str(meteorological_data_path / f"{date_time}_S_dn.tif"),
    "S_dn_24": str(meteorological_data_path / f"{date_time}_S_dn_24.tif"),
    "emis_C": 0.99,
    "emis_S": 0.97,
    "tau_vis_C": str(base_data_path / f"{date_str}_TAU_VIS_C.tif"),
    "rho_vis_C": str(base_data_path / f"{date_str}_RHO_VIS_C.tif"),
    "rho_nir_C": str(base_data_path / f"{date_str}_RHO_NIR_C.tif"),
    "tau_nir_C": str(base_data_path / f"{date_str}_TAU_NIR_C.tif"),
    "rho_vis_S": 0.15,
    "rho_nir_S": 0.25,
    "alpha_PT": 1.26,
    "x_LAD": 1,
    "z0_soil": 0.01,
    "landcover": str(base_data_path / "WordlCover2021.tif"),
    "leaf_width": str(base_data_path / f"{date_str}_LEAF_WIDTH.tif"),
    "resistance_form": 0,
    "KN_b": 0.012,
    "KN_c": 0.0038,
    "KN_C_dash": 90,
    "R_ss": 500,
    "Rst_min": 100,
    "G_form": [[1], 0.35],
    "G_ratio": 0.35,
    "G_constant": 0,
    "G_amp": 0.35,
    "G_phase": 3,
    "G_shape": 24,
    "water_stress": 1,
    "calc_row": [1, 90],
    "row_az": 90,
}

### Run the TSEB model

In [35]:
model = PyTSEB(params)
results = model.process_local_image()

Estimating missing SZA parameter
Estimating missing SAA parameter
Estimating missing L_dn parameter
data/20230531_00e8dda8/meteo_data/20230531T11649_S_dn_24.tif image not present for parameter S_dn_24
Provide a valid S_dn_24 (Daily shortwave irradiance) value if you want to estimate daily ET
Processing...


/usr/local/lib/python3.10/dist-packages/pyTSEB/meteo_utils.py:385: RuntimeWarning: divide by zero encountered in divide
  r = epsilon * ea / (p - ea)
/usr/local/lib/python3.10/dist-packages/pyTSEB/meteo_utils.py:413: RuntimeWarning: invalid value encountered in divide
  Gamma_w = ((g * (R_d * T_A_K**2 + lambda_v * r * T_A_K)
ERROR 4: data/20230531_00e8dda8/meteo_data/20230531T11649_S_dn_24.tif: No such file or directory


Finished iterations with no valid solution
Finished iterations with no valid solution
Finished processing!
['R_n1', 'H1', 'LE1', 'G1', 'CWSI']
['data/20230531_00e8dda8/20230531_image.data/20230531_image_R_n1.tif', 'data/20230531_00e8dda8/20230531_image.data/20230531_image_H1.tif', 'data/20230531_00e8dda8/20230531_image.data/20230531_image_LE1.tif', 'data/20230531_00e8dda8/20230531_image.data/20230531_image_G1.tif', 'data/20230531_00e8dda8/20230531_image.data/20230531_image_CWSI.tif']
['data/20230531_00e8dda8/20230531_image_ancillary.data/20230531_image_R_ns1.tif', 'data/20230531_00e8dda8/20230531_image_ancillary.data/20230531_image_R_nl1.tif', 'data/20230531_00e8dda8/20230531_image_ancillary.data/20230531_image_delta_R_n1.tif', 'data/20230531_00e8dda8/20230531_image_ancillary.data/20230531_image_H_C1.tif', 'data/20230531_00e8dda8/20230531_image_ancillary.data/20230531_image_LE_C1.tif', 'data/20230531_00e8dda8/20230531_image_ancillary.data/20230531_image_LE_partition.tif', 'data/2023053